In [44]:
#!pip install openpyxl
import pandas as pd


%load_ext autoreload
%autoreload 2
def d(l):
    pd.set_option('display.max_colwidth', l)

d(10)

In [5]:
science=pd.read_excel("2025-04-02 METADATA_ science-nature_V2.xlsx",sheet_name="METADATA Science")[["ID","Date","Type","Doi"]]
nature=pd.read_excel("2025-04-02 METADATA_ science-nature_V2.xlsx",sheet_name="METADATA Nature")[["ID","Date","Type","Doi"]]

df_old=pd.read_excel("Single item merged file.xlsx", usecols=['ID_Paper','Type Recoded','Type','Date YYMMDD']) # File from Fred & Auste : Better data than in the other files (in these columns)


In [21]:
GPT_retract=pd.read_json("2025-04-03 full data with GPT4o-mini_response.json")
GPT_retract["ID"]=GPT_retract.index

In [19]:
GPT_retract[(GPT_retract["retraction_context"]=="ARTICLE")&(GPT_retract["relevant"]==False)][["ID","path","text","retraction_context","category","has_retraction"]].to_excel("2025-04-25 GPT_not_relevant.xlsx")

In [7]:
GPT_retract[(GPT_retract["retraction_context"]=="ARTICLE")&(GPT_retract["relevant"]==False)].path.value_counts()

path
Data_Nature\Not relevant\Nature1722.pdf                                                     1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science755.pdf    1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science757.pdf    1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science756.pdf    1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science891.pdf    1
                                                                                           ..
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science575.pdf    1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science615.pdf    1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science609.pdf    1
Data_Science\Not relevant\Not relevant Type Perspectives and Comments\TBD\Science576.pdf    1
Data_Science\Not relevant\Not relevant Type Perspective

In [10]:
has_retraction_final=GPT_retract[["text","ID","has_retraction","retraction_context","path"]]

In [142]:
#has_retraction_final=pd.read_csv("2025-04-02b PDF_import (docling - complete).csv")[["texts","name","has_retraction"]] # Last PDF import (This is the complete Dataset)



In [241]:
# Does this make sense ??

from ast import literal_eval

Duplication_Check_len = 5000
column_name_duplicated="8. eliminated because text duplicated"

# Create a "duplicated" column by checking for duplicates in the processed text
has_retraction_final[column_name_duplicated] = has_retraction_final.text.str[:Duplication_Check_len].duplicated()
has_retraction_final["8. eliminated because text duplicated"].value_counts()

C:\Users\mr\AppData\Local\Temp\ipykernel_8068\2230678021.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_retraction_final[column_name_duplicated] = has_retraction_final.text.str[:Duplication_Check_len].duplicated()


8. eliminated because text duplicated
False    2082
True      352
Name: count, dtype: int64

In [23]:
nature_relevance=pd.read_excel("Nature tally.xlsx")
nature_relevance.columns=nature_relevance.columns.str.replace("3.eliminated","3. eliminated")
#nature_rel_categories['Date'] = pd.to_datetime(nature_rel_categories['Date'], errors='coerce')
nature_relevance.value_counts()


elimination_columns=['1. eliminated because no pdf doc',
       '2. eliminated because no occurance of retract',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning']

nature_eliminated=nature_relevance[elimination_columns].apply(lambda x: x.str.title().value_counts())

print("Sum: ",nature_eliminated.sum(axis=1))
print("\n\n",nature_relevance["8. retained for analysis"].str.title().value_counts())
nature_eliminated.T


Sum:  Yes    1179
dtype: int64


 8. retained for analysis
No     1179
Yes     655
Name: count, dtype: int64


,Yes
1. eliminated because no pdf doc,137
2. eliminated because no occurance of retract,84
3. eliminated because editorial retraction,8
"4. eliminated because book reviews, etc.",157
"5. eliminated because scanned with other relevant doc, duplicate or other technical issue",162
6. eliminated because context/meaning,631


In [24]:
#print(nature_relevance[nature_relevance['2. eliminated because no occurance of retract']=="yes"]["ID"].to_list())

In [25]:
science_relevance=pd.read_excel("science tally.xlsx",sheet_name="final tally")
science_relevance.columns=science_relevance.columns.str.replace("retrieved ","retrieved").str.replace("3.eliminated","3. eliminated")
science_relevance.value_counts()
science_eliminated=science_relevance[elimination_columns].apply(lambda x: x.str.title().value_counts())

print("Sum: ",science_eliminated.sum(axis=1))
print("\n\n",science_relevance["8. retained for analysis"].str.title().str.replace("X","No").value_counts())
science_eliminated.T


Sum:  Yes    1018
dtype: int64


 8. retained for analysis
No     1018
Yes     377
Name: count, dtype: int64


,Yes
1. eliminated because no pdf doc,248
2. eliminated because no occurance of retract,434
3. eliminated because editorial retraction,53
"4. eliminated because book reviews, etc.",43
"5. eliminated because scanned with other relevant doc, duplicate or other technical issue",65
6. eliminated because context/meaning,175


In [26]:
# all files

df_relevance=pd.concat([science_relevance,nature_relevance])
df_relevance=pd.merge(df_relevance,has_retraction_final,left_on="ID",right_on="ID",how="outer")

df_relevance.iloc[:,3:]=df_relevance.iloc[:,3:].replace({'yes': True, 'Yes': True,"YES":True})
#df_relevance["2. eliminated because no occurance of retract"]

df_relevance[elimination_columns].apply(lambda x: x.value_counts()).T

,True
1. eliminated because no pdf doc,385
2. eliminated because no occurance of retract,518
3. eliminated because editorial retraction,61
"4. eliminated because book reviews, etc.",200
"5. eliminated because scanned with other relevant doc, duplicate or other technical issue",227
6. eliminated because context/meaning,806


In [31]:
df_relevance.columns

Index(['ID', 'Title', 'Doi', 'PDF retrieved', 'If no, why?',
       'Appears in first context document?', 'If no, why?.1',
       'Appears in second context document?',
       '1. eliminated because no pdf doc',
       '2. eliminated because no occurance of retract',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning', '7. eliminated because….',
       '8. retained for analysis', 'text', 'has_retraction',
       'retraction_context', 'path', '8. eliminated because text duplicated'],
      dtype='object')

In [53]:
# only the new files

df_relevance=pd.concat([science_relevance,nature_relevance])
df_relevance=pd.merge(df_relevance,has_retraction_final,left_on="ID",right_on="ID",how="outer")

df_relevance.iloc[:,3:]=df_relevance.iloc[:,3:].replace({'yes': True, 'Yes': True, 'YES': True})
#df_relevance["2. eliminated because no occurance of retract"]
df_relevance["retraction_revision"]=(df_relevance["2. eliminated because no occurance of retract"]==True) & (df_relevance["has_retraction"]==True)

x=df_relevance.dropna(subset="has_retraction")
x[elimination_columns].apply(lambda x: x.value_counts()).T

/var/folders/xb/hcrlz1px0q5byjc3wrssw2lm0000gn/T/ipykernel_12383/9153024.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  x[elimination_columns].apply(lambda x: x.value_counts()).T


,True
1. eliminated because no pdf doc,NaN
2. eliminated because no occurance of retract,111.0
3. eliminated because editorial retraction,60.0
"4. eliminated because book reviews, etc.",198.0
"5. eliminated because scanned with other relevant doc, duplicate or other technical issue",227.0
6. eliminated because context/meaning,806.0


In [55]:
df_relevance["has_retraction"].value_counts()


has_retraction
1.0    2434
Name: count, dtype: int64

In [205]:
df=pd.concat([science,nature])
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
from datetime import datetime
df['Year']=df['Date'].apply(lambda x: x.year if isinstance(x,datetime) else x)
df["journal"]=df.ID.str.replace(r'\d+', '', regex=True)

# Recode the 'Type' column
df['Type Recoded'] = df['Type'].replace({
    'Perspectives and commentary': 'Contribution',
    'Comments & Opinion': 'Contribution',
    'Books & Arts': 'Book Reviews',
    'News': 'Contribution',
    'Correspondence': 'Journalism',
    'Editorial': 'Journalism',
    'other': 'Journalism',
    'News & Views': 'Journalism',
    'Special Features': 'Other',
    'Not comment/opinion': 'Other',
    'Meeting report': 'Other',
    'Research Highlights': 'Other',
    None: 'Other'
})


df_old['Date'] = pd.to_datetime(df_old['Date YYMMDD'], errors='coerce')
df.dropna(subset="ID",inplace=True)
df=pd.merge(df,df_old,right_on="ID_Paper",left_on="ID",how="outer")


In [206]:
# some are different
df[df["Date_x"]!=df["Date_y"]][["ID","Date_x","Date_y"]].dropna()


,ID,Date_x,Date_y
0,Nature1,2023-06-14,2023-06-22
766,Nature1689,1898-11-08,1898-08-11
1500,Nature699,1871-06-07,1871-07-06
1537,Nature731,1879-02-10,1879-10-02
1838,Science1001,1888-06-01,1888-01-06
1854,Science1016,1884-12-09,1884-09-12


In [207]:
# some are different

df[df["Type_x"]!=df["Type_y"]][["ID","Type_x","Type_y"]].dropna()


,ID,Type_x,Type_y
121,Nature1107,Comments & Opinion,News
348,Nature1311,Comments & Opinion,News
779,Nature170,Comments & Opinion,Not comment/opinion
869,Nature1781,Editorial,Meeting report
1063,Nature304,Comments & Opinion,Not comment/opinion
1294,Nature512,Comments & Opinion,News
1304,Nature521,Comments & Opinion,Not comment/opinion
1323,Nature539,Comments & Opinion,Not comment/opinion
1551,Nature744,Comments & Opinion,News


In [134]:
df.journal.value_counts()


journal
Nature     1834
Science    1395
Name: count, dtype: int64

In [208]:
# some metadata was recoded: 
# Action: I take the recoded ones!
df[df["Type_x"]!=df["Type_y"]][["ID","Type_x","Type_y"]].dropna()

# Update Type_x only where Type_y has a value
df["Type_x"] = df["Type_x"].where(df["Type_y"].isna(), df["Type_y"])
df["Type Recoded_x"] = df["Type Recoded_x"].where(df["Type Recoded_y"].isna(), df["Type Recoded_y"])
df["Date_x"] = df["Date_x"].where(df["Date_y"].isna(), df["Date_y"])


df["confirmed_2024"]=df.ID_Paper.isna()==False
df.drop(columns=["Type_y","Date_y",'Type Recoded_y','Date YYMMDD',"ID_Paper"],inplace=True)
df.columns = df.columns.str.replace('_x', '', regex=False)

In [305]:
df.columns

Index(['ID', 'Date', 'Type', 'Doi', 'Year', 'journal', 'Type Recoded',
       'confirmed_2024', '7. eliminated because before 1960'],
      dtype='object')

In [209]:
df["Year"]=df["Date"].dt.year
df["7. eliminated because before 1960"]=df["Date"]<pd.to_datetime("01-01-1960")

In [210]:
df.dropna(subset="ID",inplace=True)

In [307]:
df

,ID,Date,Type,Doi,Year,journal,Type Recoded,confirmed_2024,7. eliminated because before 1960
0,Nature1,2023-06-22,News,10.1038/d41586-023-01969-z,2023,Nature,Journalism,True,False
1,Nature10,2023-04-27,Research Highlights,10.1038/d41586-023-01416-z,2023,Nature,Other,False,False
2,Nature100,2021-06-09,Comments & Opinion,10.1038/d41586-021-01457-2,2021,Nature,Journalism,True,False
3,Nature1000,1926-12-11,News,10.1038/118855a0,1926,Nature,Contribution,False,True
4,Nature1001,2005-09-28,News,10.1038/news050926-10,2005,Nature,Contribution,False,False
...,...,...,...,...,...,...,...,...,...
3224,Science995,1963-09-27,Perspectives and commentary,10.1126/science.141.3587.1267,1963,Science,Contribution,False,False
3225,Science996,2001-09-21,Perspectives and commentary,10.1126/science.293.5538.2203a,2001,Science,Contribution,False,False
3226,Science997,1918-03-01,Perspectives and commentary,10.1126/science.47.1209.221,1918,Science,Contribution,False,True
3227,Science998,1918-03-01,Perspectives and commentary,10.1126/science.47.1209.220,1918,Science,Contribution,False,True


In [224]:
df_final=pd.merge(df,df_relevance,on="ID")


In [225]:
columns_to_check=['3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning', "7. eliminated because before 1960"]


df_final['3-7 eliminated'] = df_final[columns_to_check].apply(lambda row: row.any(), axis=1)
df_final['3-6 eliminated'] = df_final[columns_to_check[:-1]].apply(lambda row: row.any(), axis=1)

In [242]:
#remove text duplicates
#df_final["9. retained for analysis (revised)"]=(df_final["has_retraction"]==True) & (~df_final["3-6 eliminated"]==True) & (~df_final[column_name_duplicated].dropna()==True)

# keep text duplicates
df_final["9. retained for analysis (revised)"]=(df_final["has_retraction"]==True) & (~df_final["3-6 eliminated"]==True)


df_final["9. retained for analysis (revised)"].replace({'yes': True, 'Yes': True, 'YES': True},inplace=True)


df_final["9. retained for analysis (revised)"].value_counts()

C:\Users\mr\AppData\Local\Temp\ipykernel_8068\1832403530.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final["9. retained for analysis (revised)"].replace({'yes': True, 'Yes': True, 'YES': True},inplace=True)


9. retained for analysis (revised)
False    2086
True     1143
Name: count, dtype: int64

In [244]:
df_final["8. retained for analysis"]=df_final["8. retained for analysis"].replace("No",False).replace("no",False).replace("x",False)

df_final["8. retained for analysis"].value_counts()

8. retained for analysis
False    2197
True     1032
Name: count, dtype: int64

In [282]:
df_final["NEW in the analysis"]=(df_final["8. retained for analysis"]==False) & (df_final["9. retained for analysis (revised)"]==True) & (df_final["7. eliminated because before 1960"]==False)
df_final["REMOVED from the analysis"]=(df_final["8. retained for analysis"]==True) & (df_final["9. retained for analysis (revised)"]==False) & (df_final["7. eliminated because before 1960"]==False)

df_final[df_final["NEW in the analysis"]].to_excel("2025-04-25 new articles for manual annotation of retraction context.xlsx")





In [283]:
Include_manually_annotated=pd.read_excel("2025-04-26 Manual Annotation results_Inclusion.xlsx")["ID"].to_list()


In [297]:
df_final.loc[df_final['ID'].isin(Include_manually_annotated),"9. retained for analysis (revised)"]=True
df_final.loc[(df_final['ID'].isin(Include_manually_annotated)==False) &(df_final["NEW in the analysis"]),"9. retained for analysis (revised)"]=False

#eliminate before 1960
df_final.loc[(df_final["7. eliminated because before 1960"]==True) & (df_final["9. retained for analysis (revised)"]==True),"9. retained for analysis (revised)"]=False




In [298]:
df_final["9. retained for analysis (revised)"].value_counts()

9. retained for analysis (revised)
False    2214
True     1015
Name: count, dtype: int64

In [301]:
df_final["PDF retrieved"]=df_final["PDF retrieved"].apply(lambda x: True if x=="Yes but no OCR" else x)

df_final.to_excel("2025-04-25 all.xlsx")

In [180]:
#Reorganize Column
df_final["PDF retrieved"]=df_final["PDF retrieved"].apply(lambda x: True if x=="Yes but no OCR" else x)


df_final=df_final[['ID', 'Doi', 'journal' ,'Title', 'Date', 'Year', 'Type', 'Type Recoded',
       'confirmed_2024', 
       'PDF retrieved','has_retraction',
       '1. eliminated because no pdf doc',
       '2. eliminated because no occurance of retract',
   #    'retraction_revision',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning', '7. eliminated because before 1960', '3-7 eliminated',
       '8. retained for analysis',column_name_duplicated,"9. retained for analysis (revised)"]]


In [308]:
# Check excluded articles by year --> seems fine!

x=df_final.dropna(subset="has_retraction")#["has_retraction"]
x.loc[(x["has_retraction"]==True)&(x.Year<1960)].Date.sort_values()

817    1870-03-03
1412   1870-03-03
39     1870-03-24
1411   1870-05-19
1388   1870-05-19
          ...    
659    1959-09-12
660    1959-09-12
657    1959-10-31
658    1959-10-31
655    1959-11-14
Name: Date, Length: 567, dtype: datetime64[ns]

In [309]:
df_final

,ID,Date,Type,Doi_x,Year,journal,Type Recoded,confirmed_2024,7. eliminated because before 1960,Title,...,text,has_retraction,retraction_context,path,8. eliminated because text duplicated,3-7 eliminated,3-6 eliminated,9. retained for analysis (revised),NEW in the analysis,REMOVED from the analysis
0,Nature1,2023-06-22,News,10.1038/d41586-023-01969-z,2023,Nature,Journalism,True,False,Prized dinosaur fossil returned to Brazil after controversy,...,"Picture story Prized dinosaur fossil returnedto Brazil after controversy This celebratory image shows Brazilian researchers and government officials gathered around a fossil of the dinosaur Ubirajara jubatus.The specimen, newly returned to Brazil from Germany, sparked an outcry in late 2020 afte...",1.0,ARTICLE,Data_Nature\relevant_final\Nature1.pdf,False,False,False,True,False,False
1,Nature10,2023-04-27,Research Highlights,10.1038/d41586-023-01416-z,2023,Nature,Other,False,False,Fish on dry land hint at why we blink,...,"Proc.NatlAcad.Sci.USA120, e2220404120 (2023) Mudskippers blink not by shutting eyelidsbutby retractingtheir eyes into their heads.The authorsfound that the fish blink for similar reasons to tetrapods:in particular,they blink more oftenwhenthe air gets drier, suggesting that this helps to keep th...",1.0,OTHER,Data_Nature\Not relevant\Nature10.pdf,False,True,True,False,False,False
2,Nature100,2021-06-09,Comments & Opinion,10.1038/d41586-021-01457-2,2021,Nature,Journalism,True,False,Cells or drugs? The race to regenerate the heart,...,"Meanwhile,another heart-healing strategy has emerged,drawinginspirationfrom species that,unlike humans,can regrow cardiac muscle after trauma.Researchers are seeking tolearn more about the molecules produced byzebrafish(Daniorerio)hearts astheyheal themselves- and are investigatingwhether inject...",1.0,ARTICLE,Data_Nature\relevant_final\Nature100.pdf,False,False,False,True,False,False
3,Nature1000,1926-12-11,News,10.1038/118855a0,1926,Nature,Contribution,False,True,Research Items,...,"NO.2980,VOL.118 CRANIOMETRICAL DATA OF AN IMMATURE SKULL OF A FEMALE CHIMPANZEE.—Dr.E.Warren （Ann. Natal Mus.,vol.5,pt.3，1926)has made careful measurements of an immature skull of a chimpanzee with the express purpose of placing on record data which could be used for comparison with the Taungs f...",1.0,OTHER,Data_Nature\Not relevant\Nature1000.pdf,False,True,True,False,False,False
4,Nature1001,2005-09-28,News,10.1038/news050926-10,2005,Nature,Contribution,False,False,Water walkers surf on the edge,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,Science995,1963-09-27,Perspectives and commentary,10.1126/science.141.3587.1267,1963,Science,Contribution,False,False,"Solar Energy: Introduction to the Utilization of Solar Energy. A. M. Zarem and Duane D. Erway, Eds. McGraw-Hill, New York, 1963. xii + 398 pp.\n Illus. $13.50.",...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False
3225,Science996,2001-09-21,Perspectives and commentary,10.1126/science.293.5538.2203a,2001,Science,Contribution,False,False,This Week's Letters,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False
3226,Science997,1918-03-01,Perspectives and commentary,10.1126/science.47.1209.221,1918,Science,Contribution,False,True,"Outlines of Comparative Anatomy of Vertebrates. By J. S. Kingsley, Professor of Zoology in the University of Illinois. Philadelphia,\n P. Blakiston's Sons & Co. Second Edition, Revised. 1917. Pp. 449.",...,"From extended experience as a teacher of comparative anatomy Professor Kingsley has learned that a plain diet of anatomy is unacceptable to the average college undergraduate. "" Boning"" and ""grinding"" are college syn- onyms and the undergraduate does neither gladly. Anatomy therefore in the Kings...",1.0,OTHER,Data_Science\Not relevant\Not relevant Type Perspectives and Comments\Science997.pdf,False,True,True,False,False,False
3227,Science998,1918-03-01,Perspectives and c

In [310]:
# Drop rows with missing values in the specified column and create a new DataFrame
bad_context = df_final.dropna(subset=['6. eliminated because context/meaning'])
# Select the relevant column from the new DataFrame
bad_context['6. eliminated because context/meaning']
bad_context[["ID","Doi_x","Title","Date"]+elimination_columns+['7. eliminated because before 1960']].sort_values('7. eliminated because before 1960').to_excel("2025-04-03_context eliminations.xlsx")

In [313]:
bad_context 

## DO I also have to check this?! I trust Auste here!


,ID,Date,Type,Doi_x,Year,journal,Type Recoded,confirmed_2024,7. eliminated because before 1960,Title,...,text,has_retraction,retraction_context,path,8. eliminated because text duplicated,3-7 eliminated,3-6 eliminated,9. retained for analysis (revised),NEW in the analysis,REMOVED from the analysis
1,Nature10,2023-04-27,Research Highlights,10.1038/d41586-023-01416-z,2023,Nature,Other,False,False,Fish on dry land hint at why we blink,...,"Proc.NatlAcad.Sci.USA120, e2220404120 (2023) Mudskippers blink not by shutting eyelidsbutby retractingtheir eyes into their heads.The authorsfound that the fish blink for similar reasons to tetrapods:in particular,they blink more oftenwhenthe air gets drier, suggesting that this helps to keep th...",1.0,OTHER,Data_Nature\Not relevant\Nature10.pdf,False,True,True,False,False,False
3,Nature1000,1926-12-11,News,10.1038/118855a0,1926,Nature,Contribution,False,True,Research Items,...,"NO.2980,VOL.118 CRANIOMETRICAL DATA OF AN IMMATURE SKULL OF A FEMALE CHIMPANZEE.—Dr.E.Warren （Ann. Natal Mus.,vol.5,pt.3，1926)has made careful measurements of an immature skull of a chimpanzee with the express purpose of placing on record data which could be used for comparison with the Taungs f...",1.0,OTHER,Data_Nature\Not relevant\Nature1000.pdf,False,True,True,False,False,False
8,Nature1005,1925-09-05,News,10.1038/116373a0,1925,Nature,Contribution,False,True,Research Items,...,"RECOVERY FROMPLASMOLYSIS.-It has been almost axiomatic amongst botanists to interpret recovery fromplasmolysis,when aliving cell is immersed in a hypertonic salt solution,as due to the gradual entry of the salt. When theplant cell is immersed in sugar OBSERVATIONS ON A COLLARED FLAGELLATE.-Dr. G...",1.0,OTHER,Data_Nature\Not relevant\Nature1005.pdf,False,True,True,False,False,False
10,Nature1007,2005-06-08,News & Views,10.1038/435750a,2005,Nature,Journalism,False,False,Cue for the branching connection,...,"Probably as a consequence of millions of years of coevolution, arbuscular mycorrhizal fungi rely so much on the interaction with plants that they cannot be grown in isolation. A spore buried in the ground can only ramify through the branching threads known as hyphae if it encounters a host root....",1.0,OTHER,Data_Nature\Not relevant\Nature1007.pdf,False,True,True,False,False,False
13,Nature101,2021-06-07,Research Highlights,10.1038/d41586-021-01479-w,2021,Nature,Other,False,False,A light touch changes the strength of a single atomic bond,...,"HOSTOFBIOMARKERS OFFERCLUESTO PEOPLE'SFITNESS By varying the field's direction and strength,the researchers could either strengthen or weaken the gold-carbon bond. When they strengthened thebond enough and gently retracted the AFMtip,the graphene sheet pulled away fromthe surfacetowhichitwas att...",1.0,OTHER,Data_Nature\Not relevant\Nature101.pdf,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,Science987,1921-05-13,Perspectives and commentary,10.1126/science.53.1376.451,1921,Science,Contribution,False,True,Bryozoa as Food for Other Animals,...,"RAYMOND C. OSBURN It should be added that the statoblasts of the freshwater bryozoa are often eaten by young fishes. During a survey of the fishes of Ohio, made during the past summer, statoblasts of Pectinatella and Plumatella were found among the stomacl contents of the young of the large-mout...",1.0,OTHER,Data_Science\Not relevant\Not relevant Type Perspectives and Comments\Science987.pdf,False,True,True,False,False,False
3221,Science992,1893-01-09,Perspectives and commentary,10.1126/science.ns-22.552.124.b,1893,Science,Contribution,False,True,Coyote or Bear?,...,"Brighton, Eng. 4.“Geographical Distribution of Animals,”Vol.II., p.201. Dr.Heger, however, declines to recognize the device as representing a wolf, and declares it to be a bear from ""its fangs, claws and shaggy coat,""-characteristics, by the way, also common to the wolf. He admits that ""the tail...",1.0,OTHER,Data_S

In [315]:
df_final["has_retraction"].value_counts()

has_retraction
1.0    2434
Name: count, dtype: int64

In [314]:
df_final["PDF retrieved"].value_counts()

PDF retrieved
True                    2844
No                       204
No - but no need          90
No but no need            54
No technical problem      37
Name: count, dtype: int64

In [316]:
df_final["9. retained for analysis (revised)"].value_counts()

9. retained for analysis (revised)
False    2214
True     1015
Name: count, dtype: int64

In [191]:
print("without PDF: ",df_final.shape[0]-df_final["PDF retrieved"].value_counts()[0])

without PDF:  385


C:\Users\mr\AppData\Local\Temp\ipykernel_8068\1747359325.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("without PDF: ",df_final.shape[0]-df_final["PDF retrieved"].value_counts()[0])


In [192]:
df_final['Type'].value_counts()

Type
News                           1670
Perspectives and commentary     828
News & Views                    204
Books & Arts                    163
Comments & Opinion              156
Correspondence                   98
Editorial                        57
Research Highlights              31
Special Features                 17
Not comment/opinion               4
Meeting report                    1
Name: count, dtype: int64

In [193]:
df_final.has_retraction
#.value_counts()

0       1.0
1       1.0
2       1.0
3       1.0
4       NaN
       ... 
3224    NaN
3225    NaN
3226    1.0
3227    1.0
3228    NaN
Name: has_retraction, Length: 3229, dtype: float64

In [317]:
df_final.to_excel("2025-04-25 all_Meta_Data.xlsx")

PermissionError: [Errno 13] Permission denied: '2025-04-25 all_Meta_Data.xlsx'